# Installing the needed libraries

In [24]:
!pip install langchain openai pypdf chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached tenacity-8.2.3-py3-none-any.whl.metadata (1.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 6.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.7/198.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.5/528.5 kB 8.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 13.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105

In [2]:
!pip install langchain-openai 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.4.0
    Uninstalling openai-1.4.0:
      Successfully uninstalled openai-1.4.0


In [5]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 5.9 MB/s eta 0:00:00a 0:00:01


# Setting up the env

After installing all the packages we need to set the env variable for OPEN AI API.
You can do that by doing the following:

```
export OPENAI_API_KEY="KEYGOESHERE"
```

# Loading in the model

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [32]:
# A simple prompt
llm.invoke("skill issue")

AIMessage(content="I'm sorry to hear that you are experiencing a skill issue. Can you please provide more details about the specific skill you are having trouble with so that I can offer you some guidance or assistance?", response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 9, 'total_tokens': 48}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})

In [33]:
# Load in a resume:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("resume.pdf")

In [34]:
pages = loader.load()

In [35]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

In [36]:
directory = 'index_store'
vector_index = Chroma.from_documents(pages, OpenAIEmbeddings(), persist_directory=directory)
vector_index.persist()

In [26]:
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
retriever = vector_index.as_retriever(search_type="similarity", search_kwargs={"k":6})
qa_interface = RetrievalQA.from_chain_type(llm=ChatOpenAI(), chain_type="stuff", retriever=retriever, return_source_documents=True)

In [44]:
result = qa_interface("What skills does the person have in the domain of computer science? Return each skill separated by new line. Just list the skill dont say anything else. Based on these skills create a question that tests them after listed skills that can be solved within 15 minutes")['result']

Number of requested results 6 is greater than number of elements in index 4, updating n_results = 4


In [45]:
result.split('\n')

['- Machine Learning',
 '- Python (Pandas, PyTorch, scikit-learn)',
 '- JavaScript (NodeJS, ReactJS, express)',
 '- REST API',
 '- SQL',
 '- MongoDB',
 '- High Performance Computing',
 '- Linux',
 '- Docker',
 '',
 '**Question:**',
 'Given a dataset with customer information stored in a SQL database, develop a Python script that connects to the database, extracts the necessary data, performs a machine learning task (such as clustering or classification) using PyTorch, and then visualizes the results using a React front-end. The final output should be a web application that allows users to interact with the clustering/classification results.']